In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.own import OwnSingleModel
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import plot_model

from env import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs_sw', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)

MODEL_FILE = os.path.join(SAVED_MODELS_PATH, 'own', 'single.h5')
RESULT_CSV = os.path.join(RESULTS_PATH, 'own_single.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

seq = 'words_seq'
train_x = np.array(list(train_with_seq[seq]))
val_x = np.array(list(val_with_seq[seq]))
test_x = np.array(list(test_with_seq[seq]))

y_cols = LABELS
val_ys = []
train_outputs = []
val_outputs = []
for col in y_cols:
    train_y = train_with_seq[col] + 2
    val_y = val_with_seq[col] + 2
    val_ys.append(val_y)
    train_y_onehot = to_categorical(train_y)
    val_y_onehot = to_categorical(val_y)
    train_outputs.append(train_y_onehot)
    val_outputs.append(val_y_onehot)

In [6]:
model = OwnSingleModel(max_len = train_x.shape[1], embedding = embedding)
model._model.summary()
plot_model(model._model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1123)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1123, 300)    15000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1123, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 1123, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
spatial_dr

In [7]:
model.fit(train_x, train_outputs, 
          validation_data = (val_x, val_outputs),
          model_file = MODEL_FILE
         )

Train on 105000 samples, validate on 15000 samples
Epoch 1/300
105000/105000 [==============================] - 503s 5ms/step - loss: 16.2318 - dense_1_loss: 0.7207 - dense_2_loss: 0.6603 - dense_3_loss: 0.8102 - dense_4_loss: 0.5850 - dense_5_loss: 1.0122 - dense_6_loss: 0.4067 - dense_7_loss: 0.6674 - dense_8_loss: 1.0759 - dense_9_loss: 0.6609 - dense_10_loss: 0.8854 - dense_11_loss: 0.9443 - dense_12_loss: 0.8434 - dense_13_loss: 1.0188 - dense_14_loss: 0.9278 - dense_15_loss: 1.1735 - dense_16_loss: 0.9500 - dense_17_loss: 0.8746 - dense_18_loss: 0.6948 - dense_19_loss: 0.7369 - dense_20_loss: 0.5828 - val_loss: 11.4839 - val_dense_1_loss: 0.5687 - val_dense_2_loss: 0.5384 - val_dense_3_loss: 0.6936 - val_dense_4_loss: 0.4226 - val_dense_5_loss: 0.5482 - val_dense_6_loss: 0.2736 - val_dense_7_loss: 0.4737 - val_dense_8_loss: 0.7499 - val_dense_9_loss: 0.4004 - val_dense_10_loss: 0.5034 - val_dense_11_loss: 0.5613 - val_dense_12_loss: 0.5437 - val_dense_13_loss: 0.7389 - val_dense_

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.39928, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/own/single.h5
Epoch 2/300
105000/105000 [==============================] - 495s 5ms/step - loss: 10.5380 - dense_1_loss: 0.3315 - dense_2_loss: 0.4651 - dense_3_loss: 0.5126 - dense_4_loss: 0.3910 - dense_5_loss: 0.5557 - dense_6_loss: 0.1724 - dense_7_loss: 0.4001 - dense_8_loss: 0.7384 - dense_9_loss: 0.4142 - dense_10_loss: 0.5391 - dense_11_loss: 0.5474 - dense_12_loss: 0.5177 - dense_13_loss: 0.7201 - dense_14_loss: 0.5555 - dense_15_loss: 0.8494 - dense_16_loss: 0.6860 - dense_17_loss: 0.6992 - dense_18_loss: 0.4551 - dense_19_loss: 0.5582 - dense_20_loss: 0.4293 - val_loss: 8.6831 - val_dense_1_loss: 0.2438 - val_dense_2_loss: 0.3531 - val_dense_3_loss: 0.3517 - val_dense_4_loss: 0.3261 - val_dense_5_loss: 0.4526 - val_dense_6_loss: 0.1080 - val_dense_7_loss: 0.2929 - val_dense_8_loss: 0.6308 - val_dense_9_loss: 0.3506 - val_dense_10_loss: 0.4

105000/105000 [==============================] - 496s 5ms/step - loss: 7.9125 - dense_1_loss: 0.2009 - dense_2_loss: 0.2926 - dense_3_loss: 0.2829 - dense_4_loss: 0.3051 - dense_5_loss: 0.4201 - dense_6_loss: 0.1097 - dense_7_loss: 0.2476 - dense_8_loss: 0.5758 - dense_9_loss: 0.3553 - dense_10_loss: 0.4623 - dense_11_loss: 0.4242 - dense_12_loss: 0.3990 - dense_13_loss: 0.5074 - dense_14_loss: 0.4299 - dense_15_loss: 0.6429 - dense_16_loss: 0.5404 - dense_17_loss: 0.5746 - dense_18_loss: 0.3479 - dense_19_loss: 0.4442 - dense_20_loss: 0.3497 - val_loss: 7.7319 - val_dense_1_loss: 0.1848 - val_dense_2_loss: 0.3001 - val_dense_3_loss: 0.2764 - val_dense_4_loss: 0.2818 - val_dense_5_loss: 0.4056 - val_dense_6_loss: 0.1008 - val_dense_7_loss: 0.2316 - val_dense_8_loss: 0.5539 - val_dense_9_loss: 0.3282 - val_dense_10_loss: 0.4358 - val_dense_11_loss: 0.4137 - val_dense_12_loss: 0.3923 - val_dense_13_loss: 0.4763 - val_dense_14_loss: 0.4040 - val_dense_15_loss: 0.6408 - val_dense_16_loss: 

In [8]:
model._model.load_weights(MODEL_FILE)
val_probs = model.predict(val_x)
test_probs = model.predict(test_x)

val_preds = list(map(lambda x: np.argmax(x, axis = -1), val_probs))
# val_preds = np.argmax(val_probs, axis = 1)
# test_preds = list(map(lambda x: np.argmax(x, axis = -1), test_probs))

In [9]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_ys, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.605466622247453
The 1th f1: 0.502159358188295
The 2th f1: 0.626253418681543
The 3th f1: 0.5627216442135571
The 4th f1: 0.76670829521768
The 5th f1: 0.5170212115127293
The 6th f1: 0.6424132286511315
The 7th f1: 0.7493463835096147
The 8th f1: 0.6883904199916078
The 9th f1: 0.5938356266491976
The 10th f1: 0.6806562906095373
The 11th f1: 0.7260506256647516
The 12th f1: 0.7250364915288556
The 13th f1: 0.6963565770584659
The 14th f1: 0.6591619739738395
The 15th f1: 0.687874474323913
The 16th f1: 0.46662034314991496
The 17th f1: 0.6724100280515015
The 18th f1: 0.5527347542662917
The 19th f1: 0.6730820552972366
The average f1 of val is 0.6397149911393558


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
